In [1]:
import torch
import torch.nn as nn

In [2]:
# 卷积对输入图X通过卷积核K进行卷积操作的函数
def My_conv2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y

In [7]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
print("original X: ")
print(X)
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
print("convolution kernel K: ")
print(K)
print("\nafter convolution: ")
print(My_conv2d(X, K))

original X: 
tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])
convolution kernel K: 
tensor([[0., 1.],
        [2., 3.]])

after convolution: 
tensor([[19., 25.],
        [37., 43.]])


In [9]:
# 利用简单卷积核对自己写的卷积函数的正确性进行验证
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [11]:
# 类似检测竖直边缘的卷积核
K = torch.tensor([[1.0, -1.0]])
K

tensor([[ 1., -1.]])

In [13]:
Y = My_conv2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [18]:
# 将手写的卷积操作放入自定义的卷积层
class My_Con2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, X):
        return My_conv2d(X, self.weight) + self.bias

In [22]:
# 手写一个梯度下降来对卷积核参数进行学习
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
my_conv2d = My_Con2D(kernel_size=(1, 2))

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = torch.rand(6,8)
Y = torch.rand(6,7)

learning_rate = 0.002
epoch_num = 10
for i in range(epoch_num):
    Y_hat = my_conv2d(X)
    l = ((Y_hat - Y)**2).sum()  # 计算MSE作为loss
    my_conv2d.zero_grad()  # 计算梯度
    l.backward()  # 反向传播计算梯度
    # 参数更新，具体学习过程
    my_conv2d.weight.data[:] -= learning_rate * my_conv2d.weight.grad
    print("loss of epoch"+ str(i)+": " + f'{l:.3f}')


loss of epoch0: 12.589
loss of epoch1: 11.174
loss of epoch2: 10.025
loss of epoch3: 9.092
loss of epoch4: 8.334
loss of epoch5: 7.717
loss of epoch6: 7.214
loss of epoch7: 6.805
loss of epoch8: 6.470
loss of epoch9: 6.195
